## 🚀 DipDetectorML — Presentation & Integration Notebook  
**Team WaveRiders**

This notebook shows the *presentation and integration* side of our project:
- 📉 Live BTC monitoring (CoinGecko API)
- 🤖 Monthly ML probability (from Mohammed’s Random Forest)
- ✉️ Email alert previews (HTML)
- 🗺️ Architecture flow (Mermaid)
- 📊 Stakeholder visuals (price + thresholds)
- ☁️ Cloud plan (scaffold only, no sending here)

**Author & Role:** Jessenia — Project Lead, Dashboard & Integration


## 📑 Data Dictionary — Integration Layer

- `timestamp_utc` — snapshot time (UTC)
- `symbol` — “BTC”
- `price_usd` — current price
- `change_24h_pct` — 24h percent change (fraction, e.g., −0.052 = −5.2%)
- `threshold_pct` — user-selected threshold (2, 5, 10, 15, 20)
- `triggered` — True/False (dip ≥ threshold)
- `monthly_dip_prob` — ML forecast of ≥20% monthly dip (0–1)
- `email_preview_html` — formatted alert body (preview only)


## 🧭 End-to-End Architecture (Flowchart)

```mermaid
flowchart LR
  subgraph U[Users]
    U1[Subscribe or set preferences\nemail, thresholds 2/5/10/15/20]
  end
  subgraph ING[Live Data Ingest]
    I1[Fetch BTC price\nCoinGecko API]
    I2[Compute percent change\n24h or 7d window]
  end
  subgraph AE[Alert Engine]
    A1{Dip >= threshold?}
    A2[Log check\ndata/price_log.csv]
  end
  subgraph ML[ML Pipeline]
    M1[Load historical BTC CSV]
    M2[Feature engineering\nreturns, RSI, volatility, volume]
    M3[Train Random Forest]
    M4[Evaluate and write metrics\nreports/metrics.txt]
    M5[Monthly forecast]
    M6[[Write artifact\ndata/ml_monthly_prob.json]]
  end
  subgraph EM[Email Delivery]
    E1[Build email HTML]
    E2{Delivery channel}
    E3[SMTP demo]
    E4[AWS SES future]
  end
  subgraph UI[Dashboard]
    D1[Read live snapshot]
    D2[Read monthly_dip_prob\ndata/ml_monthly_prob.json]
    D3[Render Streamlit UI\napp.py]
    D4[Show email preview and threshold status]
    D5[Visuals: price line and dip thresholds]
  end
  subgraph DS[Data Stores]
    S1[(data/price_log.csv)]
    S2[(data/ml_monthly_prob.json)]
    S3[(models/rf_monthly.pkl)]
    S4[(reports/metrics.txt)]
  end

  U1 -->|prefs: threshold, email| D3
  I1 --> I2 --> A1 --> A2 --> S1
  A1 -- Yes --> E1
  A1 -- No  --> D1
  E1 --> E2
  E2 -- SMTP --> E3 -->|send| U1
  E2 -- SES  --> E4 -. future .-> U1
  M1 --> M2 --> M3 --> M4 --> M5 --> M6 --> S2
  M3 --> S3
  S1 --> D1
  S2 --> D2
  D1 --> D3
  D2 --> D3
  D3 --> D4
  D3 --> D5


## 4) 🧰 Imports (Code)


In [2]:

## 4) 🧰 Imports (Code)
- Commented, minimal, no AWS here.

```python
# ============================
# Jessenia's Imports (Integration/Presentation)
# ============================

# Core
import os, json
from datetime import datetime, timezone

# Data & Viz
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import HTML, display

# Live snapshot
import requests

# Note: Streamlit lives in DipDetectorMLapp.py, not executed in this notebook.


SyntaxError: invalid syntax (3368979163.py, line 2)

## ⚡ Live BTC Snapshot (Code + tiny fallback)

In [3]:
def fetch_btc_snapshot():
    """Live price + 24h change from CoinGecko; fallback to demo values if rate-limited."""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "bitcoin", "vs_currencies": "usd", "include_24hr_change": "true"}
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()["bitcoin"]
        price = float(data["usd"])
        ch24 = float(data.get("usd_24h_change", 0.0)) / 100.0  # % -> fraction
    except Exception:
        price, ch24 = 56800.00, -0.034  # demo fallback
    now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")
    return {"timestamp_utc": now, "symbol": "BTC", "price_usd": price, "change_24h_pct": ch24}

snap = fetch_btc_snapshot()
pd.DataFrame([snap])


NameError: name 'datetime' is not defined

## ✉️ Email Alert Preview (no-send)
We render the alert email body and subject using today’s snapshot and a chosen threshold (demo only).

def fmt_usd(x): return f"${x:,.2f}"
def fmt_pct(x): return f"{x*100:.2f}%"

def build_subject(symbol, pct_change, price, window, threshold):
    arrow = "▼" if pct_change < 0 else "▲"
    return f"✅ DipDetectorML – {symbol} Alert: {arrow}{fmt_pct(pct_change)} in last {window} (Now {fmt_usd(price)})"

def build_html(symbol, price_now, pct_change, threshold, window, monthly_prob=None):
    prob_line = f"<li>Model forecast: <b>{monthly_prob*100:.1f}%</b> chance of ≥20% dip this month</li>" if monthly_prob is not None else ""
    color = "#d32f2f" if pct_change < 0 else "#2e7d32"
    arrow = "▼" if pct_change < 0 else "▲"
    return f"""
<!doctype html>
<html>
  <body style="font-family:Arial,sans-serif;line-height:1.55;color:#111;margin:0;padding:20px;background:#0f172a">
    <div style="max-width:720px;margin:0 auto;background:#0b1020;border:1px solid #1f2a44;border-radius:14px;padding:22px;color:#e5e7eb">
      <h2 style="color:#f59e0b;margin:0 0 10px">🚀 DipDetectorML Alert – {int(threshold)}% Dip Triggered!</h2>
      <div style="display:inline-block;padding:6px 10px;border-radius:999px;background:{color};color:#fff;font-weight:700;margin:6px 0 14px">
        {arrow} {fmt_pct(pct_change)} in last {window}
      </div>
      <ul style="margin-top:0">
        <li>Current price: <b>{fmt_usd(price_now)}</b></li>
        {prob_line}
      </ul>
      <p>📌 Accumulation playbook (example):</p>
      <ul>
        <li>5% dip → advance weekly DCA</li>
        <li>10% dip → 2× DCA</li>
        <li>20%+ dip → 3× DCA (if cash reserved)</li>
      </ul>
      <hr style="border:none;border-top:1px solid #1f2a44;margin:16px 0">
      <p style="font-size:12px;color:#94a3b8">Preview only (no email sent). Powered by WaveRiders 🌊</p>
    </div>
  </body>
</html>
""".strip()

threshold = 10
window = "24h"
symbol, price, ch24 = snap["symbol"], snap["price_usd"], snap["change_24h_pct"]
monthly_prob_demo = 0.32

subject = build_subject(symbol, ch24, price, window, threshold)
html_preview = build_html(symbol, price, ch24, threshold, window, monthly_prob_demo)

print("Subject:", subject)
display(HTML(html_preview))


## 🤖 Monthly ≥20% Dip Probability (from ML artifact)
We display the probability produced by Mohammed’s Random Forest model.  
This notebook reads `data/ml_monthly_prob.json` if available.


In [ ]:
# Try to load from data/ml_monthly_prob.json; fallback to demo value
monthly_prob = monthly_prob_demo
model_version = "rf_monthly_v1.pkl"
last_model_update = snap["timestamp_utc"]

try:
    with open("data/ml_monthly_prob.json", "r") as f:
        obj = json.load(f)
    monthly_prob = float(obj.get("monthly_dip_prob", monthly_prob))
    model_version = obj.get("model_version", model_version)
    last_model_update = obj.get("last_model_update", last_model_update)
except FileNotFoundError:
    pass

pd.DataFrame([{
    "monthly_dip_prob": monthly_prob,
    "model_version": model_version,
    "last_model_update": last_model_update
}])


## 📊 Stakeholder Visuals
- Price line with dip threshold
- “Last checks” table with “would trigger?” and subject preview


In [4]:
# Build a short demo price series around current price for the chart
dates = pd.date_range(end=pd.Timestamp.utcnow(), periods=60, freq="D")
rng = np.random.default_rng(13)
series = np.linspace(price*1.08, price*0.92, len(dates)) + rng.normal(0, price*0.006, len(dates))
df_demo = pd.DataFrame({"date": dates, "price": series})

fig = px.line(df_demo, x="date", y="price", title="BTC Price (demo)")
dip_line = price * (1 - threshold/100.0)
fig.add_hline(y=dip_line, line_dash="dot", annotation_text=f"{threshold}% dip line", annotation_position="bottom right")
fig.show()

# Last checks table (vary % around current)
rows = []
for i in range(6):
    pct = ch24 + (i-3)*0.01  # +/- 1% steps
    trig = (pct <= -threshold/100.0)
    subj = build_subject(symbol, pct, price, "24h", threshold)
    rows.append({
        "timestamp_utc": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC"),
        "price_usd": price,
        "change_24h_pct": pct,
        "threshold_pct": threshold,
        "triggered": trig,
        "subject_preview": subj
    })
pd.DataFrame(rows)


NameError: name 'pd' is not defined

## 🖥️ Dashboard Hook (Streamlit in `DipDetectorMLapp.py`)
The interactive UI lives in a separate file:
```bash
streamlit run DipDetectorMLapp.py


## 🧭 Roadblocks & Next Steps
**Roadblocks**
- API 403/422 handling and free-tier rate limits
- Email sending not fully automated in cloud
- Time to polish dashboard deployment

**Next Steps**
- Read ML probability from artifact on refresh
- Connect live dip logic to dashboard “Triggered” state
- Switch to SES for sending; add subscribers CSV & per-user thresholds


## 🙏 Thanks

- 😎 Maurice (Mentor) 💡 — for patient guidance and encouragement.  
- DS Peer Tutor Fari 🤝 — for hands-on support during roadblocks.  
- Farukh & Gaurav (Instructors) 🎓 — for teaching and feedback.  
- CoinGecko 🦎 — for free API access.  